# **아트테리어 2021 데이터 작업 및 Site 구축**
```
데이터 수집 => Pre Process => 가설 => 
가설 실험 설계 => 가설 검증 및 모델링 => 마무리

stores.csv  =>  data/stores_part1.csv (38개 매장),  data/stores_part_rest.csv
```


## **1 PreProcess**
데이터 전처리 및 필요한 부분 필터링

In [1]:
folder_name = "dataNew"
import pandas as pd
table = pd.read_csv(f'{folder_name}/stores.csv')
# table = table.iloc[:, [5,13,12,7,8,3,4,14,15]] # 6,
table.iloc[90:93,:]

,No,상호,업종,사업장 주소,신청인,연락처,개선희망사항
90,94,해남식당,간이음식,봉천동 464-8,박납심,010-2491-5636,간판 등
91,95,호남치킨,치킨,봉천동 464-8,천영석,010-4617-4048,간판 등
92,96,CODE101 커피,카페,보라매로 7 1층,박주현,010-9248-1443,NaN


In [2]:
# 29 및낮 : 피부관리실 주소 재확인 필요
# 60 중앙철물
# 65 100(일공공)
# 구주소, 새주소 입력시 동일한 경도위도 결과값 확인하기
from tqdm import tqdm
from data.arterior import naver_address
loglat = [naver_address('서울 관악구 '+_)[-2:] for _ in tqdm(table['사업장 주소'])]
loglat = pd.DataFrame(loglat, columns=['lon', 'lat'])
table = table.merge(loglat, left_index=True, right_index=True)
table.columns = ["index","title","part",'address','name','telephone','order','lon','lat']
table.to_csv(f'{folder_name}/stores_new.csv', index=None)
table = pd.read_csv(f'{folder_name}/stores_new.csv')
table.head(3)

,no,title,part,address,name,telephone,order,lon,lat
0,1,복돈이부추삼겹살 2호점,한식,"남현1길 68-14, 2층 나호",장동식,010-4600-9750,NaN,126.981145,37.475174
1,2,WD엘림부동산공인중개사사무소,"부동산중개업, 컨설팅","봉천동 858-12, 101호",박주현,010-5220-9792,NaN,126.953315,37.481620
2,3,은송부동산중개사사무소,"부동산중개업, 컨설팅","봉천동 858-7, B03호",은춘선,010-8945-9792,NaN,126.953402,37.481371


## **2 Folium Visualization**
```java
! pip install folium
! pip install git+https://github.com/python-visualization/branca.git@master
```
데이터 전처리 및 필요한 부분 필터링
http://localhost:8888/lab?token=82d41f14a2741de955234ece026e0543b1ddacb251e336a7

In [3]:
import folium
lat, long = table['lat'].mean(), table['lon'].mean() # 지도의 중심 계산하기
instance_map = folium.Map([lat,long], zoom_start=14)
for i in table.index:
    sub_lat, sub_long, title = table.loc[i,'lat'], table.loc[i,'lon'], table.loc[i,'title']
    folium.CircleMarker([sub_lat, sub_long], color='blue', radius = 7, tooltip = title).add_to(instance_map)
# instance_map.save('data/map_test.html')
instance_map 

In [4]:
import folium
from folium import plugins

def folium_map(table, title='label', tooltip="name", opacity=0.3):
    # 'white', 'beige', 'pink', 'lightgray','lightgreen', 'lightblue','lightred',  
    # color_map = ['gray','red', 'purple', 'lightred',  'black',  'green',  'darkblue', 
    #    'blue', 'darkgreen',  'darkpurple', 'darkred', 'orange', 'cadetblue'] * 10
    color_map = ['red', 'blue', 'purple', 'orange', 'green', 'black'] * 30
    color_map = {_:color_map[no]  for no, _ in enumerate(sorted(set(table[title].to_list())))}
    table = table.reset_index(drop=True)
    instance_map = folium.Map(  # tiles = 'Stamen Watercolor',
        [table['lat'].mean(), table['lon'].mean()], zoom_start = 14)
    for _ in range(len(table)):
        # if _ != 44: # 중복기재로 인한 제외 인덱스 데이터 
        color = color_map[table.loc[_,title]]
        title_name = str(_) + "," + str(table.loc[_,title])
        # Label 에서 "c" 가 붙었는지 내용 확인하기
#         if title_name.find('c') != -1:
#             title_name    = title_name.split(",")[-1]
#             opacity_value = opacity  # 이미 분류작업이 끝난경우
#         else: 
#             title_name    = title_name.split(",")[0]
#             opacity_value = 1
        icon_style = plugins.BeautifyIcon(
            number = title_name,
            icon_shape = "marker", inner_icon_style = 'margin-top:0;',
            border_color = color, text_color = color, border_width=2,
        )
        folium.Marker([
            table.loc[_,'lat'], table.loc[_,'lon']], #opacity = opacity_value,
            # tooltip = title_name,          
            # tooltip = str(_)+":"+table.loc[_,tooltip]+":"+table.loc[_,'category'],
            icon = icon_style).add_to(instance_map)
    return instance_map

## **1 K means Classification**
```
(3 * 20) + (4 * 13) = 112
3*7 + 4*4 = 37
3*7 + 4*4 = 37
3*6 + 4*5 = 38
```

In [5]:
from collections import Counter
from sklearn.cluster import KMeans
import matplotlib.colors as mcolors

kmeans = KMeans(n_clusters = 4, init ='k-means++') # 33명 구분 결과값 특정
kmeans.fit(table.loc[:, ['lon', 'lat']])           # K means 학습모델 완성
table['label'] = kmeans.fit_predict(table.loc[:, ['lon', 'lat']])
table['label'] = list(map(lambda x : x+1 , table['label'])) # 0 을 지우고 1개씩 더하기
color_names    = list(mcolors.cnames.keys())
table['color'] = [color_names[_]  for _ in table["label"]]
print(Counter(table['label']))

Counter({4: 37, 2: 34, 1: 23, 3: 9})


## **01 Parts 1**
외각지역 먼저 필터링

In [6]:
# 외진 지역 먼저 filtering : 26 개 [ (3X2) + (4X5)  (+추가) + (3X4) ]
label_number = table[table['title'] == '깡돈']['label'].values[0]
table[table['label'] == label_number].shape[0]     
index_part = list(table[table['label'] == label_number].index)
len(index_part)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
store_part = ['은천회관','참좋은행복','영빈제과','델리명과','FC프리즘','래몬헤어','마이리빈',
              '무지개 인테리어','용국이네 숯불갈비','반짝반짝윤','글로리아','으뜸할인마트'] # 12개 추가
index_part += [table[table['title'] == _].index.values[0]   for _ in store_part]
table_one = table.iloc[index_part, :].reset_index(drop=True)
table_one.to_csv('data/stores01.csv', index=None)
table_one.tail(3) # 38개 매장

## **02 Parts 2, 3**
나머지 지역 필터링 2개 구역 22명: 3X7 + 4X4 = 37  

In [ ]:
table_rest = table.drop(index_part).reset_index(drop=True)
table_rest.to_csv('data/stores023.csv', index=None)
table_rest.tail(3)

In [ ]:
kmeans = KMeans(n_clusters = 22, init ='k-means++') # 33명 구분 결과값 특정
kmeans.fit(table_rest.loc[:, ['lon', 'lat']])           # K means 학습모델 완성
table_rest['label'] = kmeans.fit_predict(table_rest.loc[:, ['lon', 'lat']])
table_rest['label'] = list(map(lambda x : x+1 , table_rest['label'])) # 0 을 지우고 1개씩 더하기
color_names    = list(mcolors.cnames.keys())
table_rest['color'] = [color_names[_]  for _ in table_rest["label"]]
print(Counter(table_rest['label']))

In [ ]:
%matplotlib inline
from matplotlib import rcParams
from matplotlib import pyplot as plt
import matplotlib
import string
matplotlib.rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False
fig, ax = plt.subplots(figsize=(13, 13))
table_rest.plot(kind = 'scatter',
              x = 'lon', 
              y = 'lat', 
              s = 50, # marker size
              c = table_rest['color'],
              ax = ax) # marker color by group
for _ in table_rest.index:
    ax.annotate(table_rest.loc[_]['title'], 
                (table_rest.loc[_]['lon'], table_rest.loc[_]['lat']), 
                fontsize=10)
plt.title('관악아트테리어 Map', fontsize=20)
plt.xlabel('위도', fontsize=14)
plt.ylabel('경도', fontsize=14)
plt.show()